# 01 · Download & Clean

**Goal:** Ingest latest NHSE monthly A&E and ECDS files, standardize columns, and export tidy CSVs to `data/processed/`.

> Business framing: Reduce 12‑hour waits and lift 4‑hour performance without extra headcount. This notebook prepares data for KPI analysis.

In [ ]:
# Imports
import os, pathlib
import pandas as pd

RAW = pathlib.Path('../data/raw')
OUT = pathlib.Path('../data/processed')
OUT.mkdir(parents=True, exist_ok=True)

# TODO: Place the downloaded NHS England files in ../data/raw/
# Examples (filenames vary by month release):
#   A&E_Attendances_Emergency_Admissions_2025-06.xlsx
#   ECDS_Monthly_Supplementary_2025-06.xlsx

def load_monthly_ae(filepath: str) -> pd.DataFrame:
    """Parse provider-level A&E monthly metrics; return tidy DataFrame.
    Adjust sheet names/columns as per the specific release.
    """
    # Example skeleton — edit to match exact columns
    df = pd.read_excel(filepath, sheet_name=0, header=0)
    # Standardize column names (example placeholders)
    df = df.rename(columns={
        'Provider Code':'provider_code',
        'Provider Name':'provider_name',
        'Month':'month_date',
        'Attendances (all types)':'total_attendances',
        'Within 4 hours':'within_4h_count',
        'Emergency admissions':'emergency_admissions',
        'Over 4h after DTA':'over4h_after_dta'
    })
    # Parse dates
    if 'month_date' in df.columns:
        df['month_date'] = pd.to_datetime(df['month_date']).dt.to_period('M').dt.to_timestamp()
    return df[['provider_code','provider_name','month_date','total_attendances','within_4h_count','emergency_admissions','over4h_after_dta']].dropna(subset=['provider_code'])

def load_ecds(filepath: str) -> pd.DataFrame:
    """Parse ECDS supplementary (Type 1/2 attendances + 12h-from-arrival)."""
    df = pd.read_excel(filepath, sheet_name=0, header=0)
    df = df.rename(columns={
        'Provider Code':'provider_code',
        'Provider Name':'provider_name',
        'Month':'month_date',
        'Type 1+2 attendances':'type12_attendances',
        '12h from arrival':'arrivals_12h_or_more',
        'Age band':'age_band',
        'Frailty (CFS)':'cfs_band'
    })
    if 'month_date' in df.columns:
        df['month_date'] = pd.to_datetime(df['month_date']).dt.to_period('M').dt.to_timestamp()
    keep = ['provider_code','provider_name','month_date','type12_attendances','arrivals_12h_or_more','age_band','cfs_band']
    return df[[c for c in keep if c in df.columns]].dropna(subset=['provider_code'])

# Example execution (uncomment and set filenames once files are in data/raw/)
# ae = load_monthly_ae(RAW/'A&E_Attendances_Emergency_Admissions_2025-06.xlsx')
# ecds = load_ecds(RAW/'ECDS_Monthly_Supplementary_2025-06.xlsx')
# ae.to_csv(OUT/'monthly_ae.csv', index=False)
# ecds.to_csv(OUT/'ecds_monthly.csv', index=False)

print('Ready: place source files in data/raw and uncomment loaders to export processed CSVs.')
